In [1]:
import os, random, json, re, sys
sys.path.append("../scripts/")
from collections import defaultdict
from pprint import pprint
from tqdm import tqdm
from helper import *

In [ ]:
lila_src_files = os.listdir("../data/lila/all/")

# Take a look at categories (background knowledge, math capability, language complexity)
categories = defaultdict(lambda: defaultdict(list))
for f in lila_src_files:
    j = json.load(open(f"../data/lila/all/{f}", "r"))
    if len(j['Categories']) == 0: 
        categories["NA"][0].append(f)
        continue
    for k, v in j['Categories'][0].items():
        categories[k][v].append(f)
#pprint(categories)
tmp = categories.items()
for k, v in tmp:
    categories[k] = {_k:_v for _k, _v in sorted(v.items())}
#json.dump(categories, open("../data/lila/categories.json", "w"), indent=4)

In [19]:
datasets = []

iid_test = [json.loads(l) for l in open("../data/lila/multi/iid/test.json", "r").readlines()]
print(len(iid_test))
iid_datasets = set([ex["dataset"] for ex in iid_test])

ood_test = [json.loads(l) for l in open("../data/lila/multi/ood/test.json", "r").readlines()]
print(len(ood_test))
ood_datasets = set([ex["dataset"] for ex in ood_test])

31365
8950


In [135]:
# Filter iid_test and ood_test for examples whose Output Answer is a single number 
iid_selected_ids, ood_selected_ids = [], []
for i, ex in enumerate(iid_test):
    if len(ex['Output Answer']) > 1: continue
    try: o = json.loads(ex['Output Answer'][0])
    except json.decoder.JSONDecodeError: continue
    if isinstance(o, float)or isinstance(o, int):
        iid_selected_ids.append(i)
for i, ex in enumerate(ood_test):
    if len(ex['Output Answer']) > 1: continue
    try: o = json.loads(ex['Output Answer'][0])
    except json.decoder.JSONDecodeError: continue
    if isinstance(o, float)or isinstance(o, int):
        ood_selected_ids.append(i)
print(len(iid_selected_ids) + len(ood_selected_ids))

6366


In [136]:
json.dump({
             "iid_test": iid_selected_ids,
             "ood_test": ood_selected_ids   
            },
          open("../data/lila/single_number_ans_ids.json", "w")
        )

In [133]:
answers = [ex['Output Answer'][0] for ex in iid_test if len(ex['Output Answer'])==1]
print(len(answers))

number_matched = []
number_unmatched = []
list_answers = []
non_json = []
for a in tqdm(answers):
    try: o = json.loads(a)
    except json.decoder.JSONDecodeError:
        try: o = json.loads("[{}]".format(", ".join(a.split())))
        except json.decoder.JSONDecodeError:
            non_json.append(a)
            continue
    # It appears that json.loads handles regex. It even matches numbers like 1e-4
    #if isinstance(o, str):
    #    number = re.fullmatch(r'(-?\d*\.?\d+)', a)
    #    if number is None: number_unmatched.append(a)
    #    else: number_matched.append(number.group())
    if isinstance(o, float)or isinstance(o, int):
        number_matched.append(o)
    else:
        list_answers.append(o)
print(
    len(number_matched), 
    len(number_unmatched), 
    len(non_json), 
    len(list_answers)
)
#json.dump(non_json, open("../data/lila/iid_test_nonjson_answers.json", "w"), indent=4)


100%|██████████| 27771/27771 [00:00<00:00, 109254.57it/s]

5342 0 22281 148


In [144]:
def create_in_context_ex(template, template_arg, ex_separator="\n\n"):
    try:
        return template.format(**template_arg) + ex_separator
    except KeyError:
        print("The question template and template args doesn't match!")
        raise

In [163]:
# Create 10 in-context examples
ids = random.sample(iid_selected_ids, 10)
print(ids)
template = (
    "Question: {Input}"
    "\n"
    "Program: {Output Program}"
    "\n"
    "Answer: {Output Answer}"
)

in_context_ex = ""
for i in ids:
    ex = iid_test[i]
    in_context_ex += create_in_context_ex(
        template,
        {
            "Input": ex["Input"],
            "Output Answer": ex["Output Answer"][0], 
            "Output Program": "```{}```".format(ex["Output Program"][0]),
        }
    )
print(in_context_ex)

[402, 2662, 14043, 503, 14983, 21548, 14903, 9592, 309, 13751]
Question: a dishonest shopkeeper professes to sell pulses at the cost price , but he uses a false weight of 920 gm . for a kg . his gain is … % .
Program: ```n0 = 920.0
t0 = 1.0 + 4.0
t1 = t0 * 2.0
t2 = t1 * 100.0
t3 = n0 / t2
t4 = 1 / t3
t5 = t4 - 1.0
answer = t5 * 100.0
print(answer)```
Answer: 8.695652173913038

Question: Let t(u) = u**3 - 12*u**2 - 55*u + 45. Give t(15).
Program: ```from sympy import *
u = symbols("u")
def t(u):
	return u**3 - 12*u**2 - 55*u + 45
print(t(15))```
Answer: -105

Question: -29 - (-21 + 8) - (-5 + 1)
Program: ```from sympy import *
print(-29 - (-21 + 8) - (-5 + 1))```
Answer: -12

Question: You are 4 times as old as your sister . In 8 years time you will be twice as old as your sister . What are your ages now ?
Program: ```from sympy import *
x, y= symbols('x, y')
answer = solve([y -( 4*y),y + 8 -( 2 *( y + 8 ))],(x,y))
print(answer)```
Answer: 4

Question: Daisy bought a bag of potatoes tha

In [164]:
with open("../data/lila/single_number_ans_in_ctx_ex.txt", "w") as f:
    f.write(in_context_ex)

### Draft

In [ ]:
from sympy.parsing.latex import parse_latex
from sympy import *
#s = "Multiplying both sides by $(x - 4)(2x + 1),$ we get\n\\[(2x + 3)(2x + 1) - (2x - 8)(x - 4) = (x - 4)(2x + 1).\\]This simplifies to $31x - 25 = 0,$ so $x = \\boxed{\\frac{25}{31}}.$"
for s in non_json[497:518]:
    s = normalize_final_answer(s)
    #print(s)
    s = parse_latex(r'{}'.format(s))
    #print(s, type(s))
    s = simplify(s)
    print(s, type(s))


In [81]:
s = "$\\text{51b}_{22}$"
s = normalize_final_answer(s)
s = parse_latex(r'{}'.format(s))
s

51*b_{22}